In [31]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import scipy
import re
import io

# Задача 3.1 cost equation

In [26]:
data2 = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/Electricity.csv')
data2.shape

(158, 8)

In [27]:
mod31 =smf.ols(formula='np.log(cost) ~ np.log(q)+I(np.log(q)**2)+np.log(pl)+np.log(pk)+np.log(pf)', data=data2)
Cost_eq31 = mod31.fit()
Cost_eq31.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           np.log(cost)   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                     3880.
Date:                Fri, 31 Mar 2023   Prob (F-statistic):          2.45e-158
Time:                        14:44:14   Log-Likelihood:                 91.515
No. Observations:                 158   AIC:                            -171.0
Df Residuals:                     152   BIC:                            -152.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -6.7387      0.706     -9.541      0.000      -8.134      -5.343
np.log(q)             0.4030      0.032     12.734      0.000       0.340       0.466
I(np.log(q) ** 2)     0.0304      0.002     14.024      0.000       0.026       0.035
np.log(pl)            0.1461      0.070      2.073      0.040       0.007       0.285
np.log(pk)            0.1571      0.058      2.721      0.007       0.043       0.271
np.log(pf)            0.6847      0.043     16.043      0.000       0.600       0.769
==============================================================================
Omnibus:                        2.342   Durbin-Watson:                   1.798
Prob(Omnibus):                  0.310   Jarque-Bera (JB):                2.078
Skew:                           0.109   Prob(JB):                        0.354
Kurtosis:                       3.518   Cond. No.                     4.99e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.99e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [28]:
alpha = 0.01 
print('Уровень значимости alpha: ', alpha)
DF = int(Cost_eq31.df_resid)
t_crit = scipy.stats.t.ppf(1-alpha/2, DF)
print('Критическое значение t_crit: ', t_crit.round(3))

Уровень значимости alpha:  0.01
Критическое значение t_crit:  2.609


In [49]:
table2 = pd.read_html(Cost_eq31.summary().tables[1].as_html(), header=0, index_col=0)[0] 
table2['significance'] = table2.apply(lambda x : 'Значим' if np.abs(x['t']) > t_crit else 'Незначим', axis = 1 )
table2

,coef,std err,t,P>|t|,[0.025,0.975],significance
Intercept,-6.7387,0.706,-9.541,0.000,-8.134,-5.343,Значим
np.log(q),0.4030,0.032,12.734,0.000,0.340,0.466,Значим
I(np.log(q) ** 2),0.0304,0.002,14.024,0.000,0.026,0.035,Значим
np.log(pl),0.1461,0.070,2.073,0.040,0.007,0.285,Значим
np.log(pk),0.1571,0.058,2.721,0.007,0.043,0.271,Значим
np.log(pf),0.6847,0.043,16.043,0.000,0.600,0.769,Значим


# 3.1.1 Гипотеза 1

$$ H_0: \beta_{pf} + \beta_{pl} + \beta_{pk} = 1 $$

$$$$
Интерпретация: Постоянная отдача  от  $log(pl), log(pk)$ и $log(pf)$ 

In [38]:
print(Cost_eq31.f_test("np.log(pl)+np.log(pk)+np.log(pf) = 1"))

<F test: F=0.014541184876268065, p=0.9041775484872097, df_denom=152, df_num=1>


In [35]:
alpha = 0.01 
print('Уровень значимости alpha: ', alpha)
dfn = int(1) 
dfd = int(Cost_eq31.df_resid) 
print("J =", dfn,", ", "n-k-1 =", dfd)
F_crit = scipy.stats.f.ppf(1-alpha, dfn, dfd)
print("F-critical: ", F_crit.round(3))

Уровень значимости alpha:  0.01
J = 1 ,  n-k-1 = 152
F-critical:  6.805


$$$$ Гипотеза 1 не отвергается

# 3.1.2 Гипотеза 2

$$ H_0:  \beta_{pl} = \beta_{pk} $$

$$$$
Интерпретация: отдача  от  $log(pl)$ и отдача  от $log(pk)$ равны

In [39]:
print(Cost_eq31.f_test("np.log(pl) = np.log(pk)")) 

<F test: F=0.020086275898526424, p=0.8874840609151121, df_denom=152, df_num=1>


In [40]:
alpha = 0.01
print('Уровень значимости alpha: ', alpha)
dfn = int(1) 
dfd = int(Cost_eq31.df_resid) 
print("J =", dfn,", ", "n-k-1 =", dfd)
F_crit = scipy.stats.f.ppf(1-alpha, dfn, dfd)
print("F-critical: ", F_crit.round(3))

Уровень значимости alpha:  0.01
J = 1 ,  n-k-1 = 152
F-critical:  6.805


$$$$ Гипотеза 2 не отвергается

# 3.1.3 Гипотеза 3

$$ H_0:  \beta_{pf} = \beta_{pl} = \beta_{pk}$$

$$$$
Интерпретация: отдача  от  $log(pf)$, отдача  от $log(pl)$ и отдача  от $log(pk)$ равны

In [41]:
print(Cost_eq31.f_test("np.log(pf)=np.log(pl)=np.log(pk)")) 

<F test: F=26.417080686252184, p=1.4240695015055136e-10, df_denom=152, df_num=2>


In [43]:
alpha = 0.01 
print('Уровень значимости alpha: ', alpha)
dfn = int(2) 
dfd = int(Cost_eq31.df_resid) 
print("J =", dfn,", ", "n-k-1 =", dfd)
F_crit = scipy.stats.f.ppf(1-alpha, dfn, dfd)
print("F-critical: ", F_crit.round(3))

Уровень значимости alpha:  0.01
J = 2 ,  n-k-1 = 152
F-critical:  4.748


$$$$ Гипотеза 3 отвергается

# 3.2 output equation

In [44]:
data3 = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/Labour.csv')
data3.shape

(569, 4)

In [45]:
mod32 =smf.ols(formula='np.log(output)~np.log(capital)+np.log(labour)+np.log(wage)', data=data3)
Output_eq31 = mod32.fit()
Output_eq31.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         np.log(output)   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.888
Method:                 Least Squares   F-statistic:                     1499.
Date:                Fri, 31 Mar 2023   Prob (F-statistic):          1.65e-268
Time:                        15:09:58   Log-Likelihood:                -279.62
No. Observations:                 569   AIC:                             567.2
Df Residuals:                     565   BIC:                             584.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -5.0073      0.221    -22.649      0.000      -5.442      -4.573
np.log(capital)     0.1493      0.015     10.141      0.000       0.120       0.178
np.log(labour)      0.7204      0.019     37.487      0.000       0.683       0.758
np.log(wage)        0.9214      0.058     16.001      0.000       0.808       1.034
==============================================================================
Omnibus:                      252.010   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3723.965
Skew:                           1.547   Prob(JB):                         0.00
Kurtosis:                      15.145   Cond. No.                         82.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [46]:
alpha = 0.05 
print('Уровень значимости alpha: ', alpha)
DF = int(Cost_eq31.df_resid)
t_crit = scipy.stats.t.ppf(1-alpha/2, DF)
print('Критическое значение t_crit: ', t_crit.round(3))

Уровень значимости alpha:  0.05
Критическое значение t_crit:  1.976


In [50]:
table2 = pd.read_html(Output_eq31.summary().tables[1].as_html(), header=0, index_col=0)[0]
table2['significance'] = table2.apply(lambda x : 'Значим' if np.abs(x['t']) > t_crit else 'Незначим', axis = 1 )
table2

,coef,std err,t,P>|t|,[0.025,0.975],significance
Intercept,-5.0073,0.221,-22.649,0.0,-5.442,-4.573,Значим
np.log(capital),0.1493,0.015,10.141,0.0,0.120,0.178,Значим
np.log(labour),0.7204,0.019,37.487,0.0,0.683,0.758,Значим
np.log(wage),0.9214,0.058,16.001,0.0,0.808,1.034,Значим


# 3.2.1 Гипотеза 1

$$ H_0:  \beta_{capital} + \beta_{labour} + \beta_{wage} = 1 $$

$$$$Интерпретация: Постоянная отдача от масштаба

In [51]:
A = Output_eq31.f_test("np.log(capital)+np.log(labour)+np.log(wage) = 1")
print(A.fvalue)
print(A.pvalue)
print(A.df_denom)
print(A.df_num)
print(Output_eq31.f_test("np.log(capital)+np.log(labour)+np.log(wage) = 1")) 

198.5869331584923
7.280959454662307e-39
565.0
1.0
<F test: F=198.5869331584923, p=7.280959454662307e-39, df_denom=565, df_num=1>


In [52]:
type(Output_eq31.f_test("np.log(capital)+np.log(labour)+np.log(wage) = 1"))

statsmodels.stats.contrast.ContrastResults

In [53]:
alpha = 0.05
print('Уровень значимости alpha: ', alpha)
dfn = int(1) 
dfd = int(Output_eq31.df_resid) 
print("J =", dfn,", ", "n-k-1 =", dfd)
F_crit = scipy.stats.f.ppf(1-alpha, dfn, dfd)
print("F-critical: ", F_crit.round(3))

Уровень значимости alpha:  0.05
J = 1 ,  n-k-1 = 565
F-critical:  3.858


In [55]:
F_stat = A.fvalue
if (F_stat > F_crit):
    print('Гипотеза 1 отвергается')
else:
    print('Гипотеза 1 не отвергается')

Гипотеза 1 отвергается


# 3.2.2 Гипотеза 2

$$ H_0:   \beta_{labour} = \beta_{wage} $$

$$$$Интерпретация: отдача от labour и отдача от wage равны

In [58]:
print(Output_eq31.f_test("np.log(labour)=np.log(wage)")) 

<F test: F=11.08097824634219, p=0.0009289780461988841, df_denom=565, df_num=1>


In [56]:
alpha = 0.05
print('Уровень значимости alpha: ', alpha)
dfn = int(1) 
dfd = int(Output_eq31.df_resid) 
print("J =", dfn,", ", "n-k-1 =", dfd)
F_crit = scipy.stats.f.ppf(1-alpha, dfn, dfd)
print("F-critical: ", F_crit.round(3))

Уровень значимости alpha:  0.05
J = 1 ,  n-k-1 = 565
F-critical:  3.858


In [59]:
F_stat = A.fvalue
if (F_stat > F_crit):
    print('Гипотеза 2 отвергается')
else:
    print('Гипотеза 2 не отвергается')

Гипотеза 2 отвергается


# 3.2.3 Гипотеза 3

$$ H_0:   \beta_{capital} = \beta_{labour} = \beta_{wage} $$

$$$$Интерпретация: отдача от capital, отдача от labour и отдача от wage равны

In [61]:
print(Output_eq31.f_test("np.log(capital)=np.log(labour)=np.log(wage)")) 

<F test: F=192.36687126560264, p=1.9105516323871696e-64, df_denom=565, df_num=2>


In [64]:
alpha = 0.05
print('Уровень значимости alpha: ', alpha)
dfn = int(2) 
dfd = int(Output_eq31.df_resid) 
print("J =", dfn,", ", "n-k-1 =", dfd)
F_crit = scipy.stats.f.ppf(1-alpha, dfn, dfd)
print("F-critical: ", F_crit.round(3))

Уровень значимости alpha:  0.05
J = 2 ,  n-k-1 = 565
F-critical:  3.012


In [63]:
F_stat = A.fvalue
if (F_stat > F_crit):
    print('Гипотеза 3 отвергается')
else:
    print('Гипотеза 3 не отвергается')

Гипотеза 3 отвергается
